<h5> Dataset Description </h5>
The Data description is as follows:

1. diagnosis: The diagnosis of breast tissues (1 = malignant, 0 = benign) where malignant denotes that the disease is harmful
2. mean_radius: mean of distances from center to points on the perimeter
3. mean_texture: standard deviation of gray-scale values
4. mean_perimeter: mean size of the core tumor
5. mean_area: mean area of the core tumor
6. mean_smoothness: mean of local variation in radius lengths


All feature values are recoded with four significant digits.



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.model_selection import train_test_split


### Carregando o dataset

In [ ]:
#data = pd.read_csv('Breast_cancer_data.csv')
data = pd.read_csv('data.csv')
data = data.drop('id', axis=1)
data['diagnosis'] = data['diagnosis'].map({'M':1, 'B':0})
data

### Checando se existem valores nulos

In [ ]:
## check null entries
data.isnull().sum()

### Removendo ítens duplicados

In [ ]:
## remove duplicate entries
data.drop_duplicates(inplace = True)
data

### Porque temos Raio, Perímetro e Área como features independetes? Não são features correlacionadas? Vamos investigar!

## Perímetro X Raio

In [ ]:
print(f"Correlação entre o perímetro médio e o raio médio: {data['perimeter_mean'].corr(data['radius_mean'])}")

plt.scatter(data.perimeter_mean, data.radius_mean)
plt.xlabel("Perímetro Médio")
plt.ylabel("Raio Médio")
plt.title("Relação entre Perímetro Médio e Raio Médio")
plt.show()

## Área x Raio

In [ ]:
print(f"Correlação entre o área média e o raio médio: {data['area_mean'].corr(data['radius_mean'])}")

plt.scatter(data.area_mean, data.radius_mean)
plt.xlabel("Área Média")
plt.ylabel("Raio Médio")
plt.title("Relação entre Área Média e Raio Médio")
plt.show()

## Perímetro x Área

In [ ]:
print(f"Correlação entre o perímetro médio e o área média: {data['perimeter_mean'].corr(data['area_mean'])}")



plt.scatter(data.perimeter_mean, data.area_mean)
plt.xlabel("Perímetro Médio")
plt.ylabel("Área Média")
plt.title("Relação entre Perímetro Médio e Área Média")
plt.show()

## Vamos então descartar a coluna "Perímetro" e a coluna "Área"
- Estas duas colunas não complementam a variabilidade dos dados e só dificultarão o treinamento do modelo
- Retirando essas colunas nós também retiramos a necessidade de obtê-las para uma inferência futura

In [ ]:
data.drop(['perimeter_mean'], axis=1, inplace=True)
data.drop(['area_mean'], axis=1, inplace=True)
data

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

## Tudo certo! Agora vamos separar os dados de treinamento e de validação

In [ ]:
from sklearn.model_selection import train_test_split

X = data.iloc[:,1:].values
y = data.iloc[:,0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

print(f"O conjunto de treinamento contém {len(X_train)} ítens e o conjunto de teste contém {len(X_test)} itens.")

#### Tudo certo mesmo? Vamos dar uma olhada nos valores médios de cada coluna para tentar entender 

In [ ]:
data.mean()

### Vamos realizar uma trasnformação nos dados para que fiquem com a média próxima de zero e amplitudes mais próximas

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
pd.DataFrame(X_train, columns = data.columns[1:]).mean()

## Agora sim, tudo pronto!

### Vamos instanciar as métricas para podermos avaliar os modelos posteriormente. Relembrando:

- <span style="color: green">Acertos</span>
    - "True Positive" (TP): Previsão --> <span style="color: orange">Maligno</span> ;  Real --> <span style="color: orange">Maligno</span>
    - "True Negative" (TN): Previsão --> <span style="color: cyan">Benigno</span> ;  Real --> <span style="color: cyan">Benigno</span>
<br><br>
- <span style="color: red">Erros</span>
    - "False Positive" (FP): Previsão --> <span style="color: orange">Maligno</span> ;  Real --> <span style="color: cyan">Benigno</span>
    - "False Negative" (FN): Previsão --> <span style="color: cyan">Benigno</span> ;  Real --> <span style="color: orange">Maligno</span>

### Métricas:

- Acurácia = $\frac{T_P + T_N} {T_P + T_N + F_P + F_N}$
<br><br>
- Recall = $\frac{T_P}{T_P+F_N}$
<br><br>
- Precisão = $\frac{T_P}{T_P+F_P}$
<br><br>
- F1 - Score = $\frac{{2P_{rec}R_{ec}}}{{P_{rec}+R_{ec}}}$


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

acc = {}
rec = {}
prec = {}
f1 = {}

## Regressão linear
<img src="./images/linear_regression.png" width="300">

In [ ]:
# Importando o módulo de regressão linear 
from sklearn.linear_model import LinearRegression

# Treinando o modelo 
regr = LinearRegression()
regr.fit(X_train, y_train)


# Prevendo os resultados
y_pred = regr.predict(X_test)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]


# Analisando a qualidade do modelo
cm = confusion_matrix(y_test, y_pred)

acc['reg_linear'] = accuracy_score(y_test, y_pred)
rec['reg_linear'] = recall_score(y_test, y_pred)
prec['reg_linear'] = precision_score(y_test, y_pred)
f1['reg_linear'] = f1_score(y_test, y_pred)
print(f"Acurácia: {100*acc['reg_linear']} %")
print(f"Recall: {100*rec['reg_linear']} %")
print(f"Precisão: {100*prec['reg_linear']} %")
print(f"F1: {100*f1['reg_linear']} %")

print(f"\n\nMatriz de confusão:")
plt.figure(figsize=(5,4))
sns.set(font_scale = 1)
sns.set_style("white")
_ = sns.heatmap(cm, cmap = 'gist_yarg_r',annot = True, fmt='d')

# Regressão logística
<img src="./images/logistic.png" width="300">

In [ ]:
# Treinando modelo
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)


# Prevendo os resultados
y_pred = classifier.predict(X_test)



# Analisando a qualidade do modelo
cm = confusion_matrix(y_test, y_pred)

acc['logistic'] = accuracy_score(y_test, y_pred)
rec['logistic'] = recall_score(y_test, y_pred)
prec['logistic'] = precision_score(y_test, y_pred)
f1['logistic'] = f1_score(y_test, y_pred)
print(f"Acurácia: {100*acc['logistic']} %")
print(f"Recall: {100*rec['logistic']} %")
print(f"Precisão: {100*prec['logistic']} %")
print(f"F1: {100*f1['logistic']} %")


print(f"\n\nMatriz de confusão:")
plt.figure(figsize=(5,4))
sns.set(font_scale = 1)
sns.set_style("white")
_ = sns.heatmap(cm, cmap = 'gist_yarg_r',annot = True, fmt='d')

# K Neighbors (vizinho mais próximo)
<img src="./images/k_nei.png" width="300">


In [ ]:
# Treinando modelo
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)

# Prevendo os resultados
y_pred = knn.predict(X_test)

# Analisando a qualidade do modelo
cm = confusion_matrix(y_test, y_pred)

acc['k_neighbors'] = accuracy_score(y_test, y_pred)
rec['k_neighbors'] = recall_score(y_test, y_pred)
prec['k_neighbors'] = precision_score(y_test, y_pred)
f1['k_neighbors'] = f1_score(y_test, y_pred)
print(f"Acurácia: {100* acc['k_neighbors']} %")
print(f"Recall: {100* rec['k_neighbors']} %")
print(f"Precisão: {100* prec['k_neighbors']} %")
print(f"F1: {100* f1['k_neighbors']} %")


print(f"\n\nMatriz de confusão:")
plt.figure(figsize=(5,4))
sns.set(font_scale = 1)
sns.set_style("white")
_ = sns.heatmap(cm, cmap = 'gist_yarg_r',annot = True, fmt='d')

# Linear SVM 
<img src="./images/SVM_linear.png" width="200">


In [ ]:
# Treinando modelo
from sklearn.svm import SVC
svm = SVC(kernel = 'linear')
svm.fit(X_train, y_train)

# Prevendo os resultados
y_pred = svm.predict(X_test)


# Analisando a qualidade do modelo
cm = confusion_matrix(y_test, y_pred)

acc['SVM_linear'] = accuracy_score(y_test, y_pred)
rec['SVM_linear'] = recall_score(y_test, y_pred)
prec['SVM_linear'] = precision_score(y_test, y_pred)
f1['SVM_linear'] = f1_score(y_test, y_pred)
print(f"Acurácia: {100*acc['SVM_linear']} %")
print(f"Recall: {100*rec['SVM_linear']} %")
print(f"Precisão: {100*prec['SVM_linear']} %")
print(f"F1: {100*f1['SVM_linear']} %")


print(f"\n\nMatriz de confusão:")
plt.figure(figsize=(5,4))
sns.set(font_scale = 1)
sns.set_style("white")
_ = sns.heatmap(cm, cmap = 'gist_yarg_r',annot = True, fmt='d')

# Kernal SVM
<img src="./images/SVM_kernal.png" width="200">


In [ ]:
# Treinando modelo
from sklearn.svm import SVC
kernel_svm = SVC(kernel = 'rbf')
kernel_svm.fit(X_train, y_train)

# Prevendo os resultados
y_pred = kernel_svm.predict(X_test)


# Analisando a qualidade do modelo
cm = confusion_matrix(y_test, y_pred)

acc['SVM_kernal'] = accuracy_score(y_test, y_pred)
rec['SVM_kernal'] = recall_score(y_test, y_pred)
prec['SVM_kernal'] = precision_score(y_test, y_pred)
f1['SVM_kernal'] = f1_score(y_test, y_pred)
print(f"Acurácia: {100*acc['SVM_kernal']} %")
print(f"Recall: {100*rec['SVM_kernal']} %")
print(f"Precisão: {100*prec['SVM_kernal']} %")
print(f"F1: {100*f1['SVM_kernal']} %")


print(f"\n\nMatriz de confusão:")
plt.figure(figsize=(5,4))
sns.set(font_scale = 1)
sns.set_style("white")
_ = sns.heatmap(cm, cmap = 'gist_yarg_r',annot = True, fmt='d')

# Naive Bayes (Redes Bayesianas)
<img src="./images/naive.png" width="200">


In [ ]:
# Treinando modelo
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Prevendo os resultados
y_pred = gnb.predict(X_test)


# Analisando a qualidade do modelo
cm = confusion_matrix(y_test, y_pred)

acc['naive_bayes'] = accuracy_score(y_test, y_pred)
rec['naive_bayes'] = recall_score(y_test, y_pred)
prec['naive_bayes'] = precision_score(y_test, y_pred)
f1['naive_bayes'] = f1_score(y_test, y_pred)
print(f"Acurácia: {100*acc['naive_bayes']} %")
print(f"Recall: {100*rec['naive_bayes']} %")
print(f"Precisão: {100*prec['naive_bayes']} %")
print(f"F1: {100*f1['naive_bayes']} %")


print(f"\n\nMatriz de confusão:")
plt.figure(figsize=(5,4))
sns.set(font_scale = 1)
sns.set_style("white")
_ = sns.heatmap(cm, cmap = 'gist_yarg_r',annot = True, fmt='d')

# Decision Tree Classifier (árvore de decisão)
<img src="./images/tree.png" width="200">


In [ ]:
# Treinando modelo
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

# Prevendo os resultados
y_pred = dtc.predict(X_test)

# Analisando a qualidade do modelo
cm = confusion_matrix(y_test, y_pred)

acc['tree'] = accuracy_score(y_test, y_pred)
rec['tree'] = recall_score(y_test, y_pred)
prec['tree'] = precision_score(y_test, y_pred)
f1['tree'] = f1_score(y_test, y_pred)
print(f"Acurácia: {100*acc['tree']} %")
print(f"Recall: {100*rec['tree']} %")
print(f"Precisão: {100*prec['tree']} %")
print(f"F1: {100*f1['tree']} %")


print(f"\n\nMatriz de confusão:")
plt.figure(figsize=(5,4))
sns.set(font_scale = 1)
sns.set_style("white")
_ = sns.heatmap(cm, cmap = 'gist_yarg_r',annot = True, fmt='d')

#  Random Forest
<img src="./images/random_forest.png" width="300">


In [ ]:
# Treinando modelo
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 10, criterion = 'entropy')
rfc.fit(X_train, y_train)

# Prevendo os resultados
y_pred = rfc.predict(X_test)

# Analisando a qualidade do modelo
cm = confusion_matrix(y_test, y_pred)



acc['random_forest'] = accuracy_score(y_test, y_pred)
rec['random_forest'] = recall_score(y_test, y_pred)
prec['random_forest'] = precision_score(y_test, y_pred)
f1['random_forest'] = f1_score(y_test, y_pred)
print(f"Acurácia: {100*acc['random_forest']} %")
print(f"Recall: {100*rec['random_forest']} %")
print(f"Precisão: {100*prec['random_forest']} %")
print(f"F1: {100*f1['random_forest']} %")


print(f"\n\nMatriz de confusão:")
plt.figure(figsize=(5,4))
sns.set(font_scale = 1)
sns.set_style("white")
_ = sns.heatmap(cm, cmap = 'gist_yarg_r',annot = True, fmt='d')

# Artificial Neural Network (ANN)
<img src="./images/NN.png" width="300">


In [ ]:
# Importando módulos
import tensorflow as tf
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential


# Criando modelo
model = Sequential()
model.add(Dense(units = 3, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 3, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compilando modelo
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Treinando o modelo
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
deep_history = model.fit(X_train, y_train, epochs=100, batch_size = 32, validation_data = (X_test, y_test), callbacks=[early_stop]                            )

model_path = 'modelo.h5'
model.save(model_path)

model_weights_path= 'modelo_weights.h5'
model.save_weights(model_weights_path) 

In [ ]:
## Load model
#model.load_weights(top_model_weights_path) 

In [ ]:
def plot_loss(history):
    historydata = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydata.plot(ylim=(0, 1.01*historydata.values.max()))
    plt.title('Loss: %.3f' % history.history['loss'][-1])
plot_loss(deep_history)

In [ ]:
# Prevendo os resultados
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)


# Analisando a qualidade do modelo
cm = confusion_matrix(y_test, y_pred)

acc['ANN'] = accuracy_score(y_test, y_pred)
rec['ANN'] = recall_score(y_test, y_pred)
prec['ANN'] = precision_score(y_test, y_pred)
f1['ANN'] = f1_score(y_test, y_pred)
print(f"Acurácia: {100*acc['ANN']} %")
print(f"Recall: {100*rec['ANN']} %")
print(f"Precisão: {100*prec['ANN']} %")
print(f"F1: {100*f1['ANN']} %")


print(f"\n\nMatriz de confusão:")
plt.figure(figsize=(5,4))
sns.set(font_scale = 1)
sns.set_style("white")
_ = sns.heatmap(cm, cmap = 'gist_yarg_r',annot = True, fmt='d')

# Métricas

##### Relembrando:
- <span style="color: green">Acertos</span>
    - "True Positive" (TP): Previsão --> <span style="color: orange">Maligno</span> ;  Real --> <span style="color: orange">Maligno</span>
    - "True Negative" (TN): Previsão --> <span style="color: cyan">Benigno</span> ;  Real --> <span style="color: cyan">Benigno</span>
<br><br>
- <span style="color: red">Erros</span>
    - "False Positive" (FP): Previsão --> <span style="color: orange">Maligno</span> ;  Real --> <span style="color: cyan">Benigno</span>
    - "False Negative" (FN): Previsão --> <span style="color: cyan">Benigno</span> ;  Real --> <span style="color: orange">Maligno</span>


- Acurácia = $\frac{T_P + T_N} {T_P + T_N + F_P + F_N}$
<br><br>
- Recall = $\frac{T_P}{T_P+F_N}$
<br><br>
- Precisão = $\frac{T_P}{T_P+F_P}$
<br><br>
- F1 - Score = $\frac{{2P_{rec}R_{ec}}}{{P_{rec}+R_{ec}}}$


In [ ]:
metrics = pd.DataFrame({
    'Modelo': acc.keys(),
    'Acurácia':  np.array(list(acc.values()))*100,
    'Recall': np.array(list(rec.values()))*100,
    'Precisão': np.array(list(prec.values()))*100,
    'F1': np.array(list(f1.values()))*100
}).sort_values(by = ['Recall'], ascending = False)
metrics.reset_index(drop = True, inplace = True)
metrics